In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.preprocessing import LabelEncoder
from itertools import product


import time

In [2]:
data = pd.read_pickle('data.pkl')


In [3]:
data.columns

Index(['month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'delta_price_lag', 'delta_revenue_lag_1', 'month', 'days',
       'item_last_sale', 'item_shop_first_sale', 'item_first_sale'],
      dtype='object')

In [4]:
data = data[[
    'month_idx',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'cat_id',
    'type_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'delta_price_lag',
    'delta_revenue_lag_1',
    'month',
    'days',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale'
]]


In [6]:
data.month_idx.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22], dtype=int8)

In [7]:
X_train = data[data.month_idx < 21].drop(['item_cnt_month'], axis=1)
y_train = data[data.month_idx < 21]['item_cnt_month']
X_valid = data[data.month_idx == 21].drop(['item_cnt_month'], axis=1)
y_valid  = data[data.month_idx == 21]['item_cnt_month']
X_test = data[data.month_idx ==22].drop(['item_cnt_month'], axis=1)


In [8]:
del data

In [9]:
import gc


In [10]:
gc.collect()

159

In [14]:

ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators = 1000,
    min_child_weight = 300,
    colsample_bytree=0.8,
    eta=0.3,
    seed=42
)

model.fit(X_train, y_train,eval_metric="rmse", eval_set=[(X_train, y_train),(X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10)

time.time() - ts


[0]	validation_0-rmse:1.80950	validation_1-rmse:1.90613
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.70775	validation_1-rmse:1.80289
[2]	validation_0-rmse:1.58279	validation_1-rmse:1.70585
[3]	validation_0-rmse:1.51410	validation_1-rmse:1.65457
[4]	validation_0-rmse:1.47662	validation_1-rmse:1.62277
[5]	validation_0-rmse:1.46153	validation_1-rmse:1.61121
[6]	validation_0-rmse:1.44260	validation_1-rmse:1.59970
[7]	validation_0-rmse:1.43051	validation_1-rmse:1.59032
[8]	validation_0-rmse:1.42283	validation_1-rmse:1.58654
[9]	validation_0-rmse:1.41733	validation_1-rmse:1.58124
[10]	validation_0-rmse:1.41368	validation_1-rmse:1.57938
[11]	validation_0-rmse:1.40986	validation_1-rmse:1.57879
[12]	validation_0-rmse:1.40620	validation_1-rmse:1.57750
[13]	validation_0-rmse:1.40425	validation_1-rmse:1.57879
[14]	validation_0-rmse:1.40176	validation_1-rmse:1.57

18.066138982772827

In [15]:
y_pred = model.predict(X_valid).clip(0,20)
y_test = model.predict(X_test).clip(0,20)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": y_test
})

submission.to_csv('xgb_submission.csv', index=False )


NameError: name 'test' is not defined